In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import lhapdf
import matplotlib.pyplot as plt

import functions

In [3]:
class A0(tf.keras.layers.Layer):
    def __init__(self, kperp2avg=.57, pperp2avg=.12, **kwargs):
        super(A0, self).__init__(name='a0')
        self.m1 = tf.Variable(1., name='m1')
        self.kperp2avg = kperp2avg
        self.pperp2avg = pperp2avg
        self.e = tf.constant(1.)
        
    def get_config(self):
        config = super().get_config().copy()
        
        config.update({
            'kperp2avg': self.kperp2avg,
            'pperp2avg': self.pperp2avg
        })
        
        return config
        
    def call(self, inputs):
        z = inputs[:, 0]
        pht = inputs[:, 1]
        ks2avg = (self.kperp2avg*self.m1**2)/(self.m1**2 + self.kperp2avg) #correct 
        topfirst = (z**2 * self.kperp2avg + self.pperp2avg) * ks2avg**2 #correct
        bottomfirst = (z**2 * ks2avg + self.pperp2avg)**2 * self.kperp2avg #correct
        exptop = pht**2 * z**2 * (ks2avg - self.kperp2avg) #correct
        expbottom = (z**2 * ks2avg + self.pperp2avg) * (z**2 * self.kperp2avg + self.pperp2avg) #correct
        last = tf.sqrt(2*self.e) * z * pht / self.m1 #correct      
        return (topfirst/bottomfirst) * tf.exp(-exptop/expbottom) * last

class Quotient(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Quotient, self).__init__()
    
    def call(self, inputs):
        if len(inputs) != 2 or inputs[0].shape[1] != 1:
            raise Exception('must be two tensors of shape (?, 1)')
        return inputs[0]/inputs[1]    

def h(model, kperp):
    m1 = model.get_layer('a0').m1.numpy()
    e = model.get_layer('a0').e.numpy()
    return np.sqrt(2*e) * (kperp/m1) * np.exp(-kperp**2/m1**2)

    
def nnq(model, x, hadronstr):
    if not hadronstr in ['nnu', 'nnd', 'nns', 'nnubar', 'nndbar', 'nnsbar']:
        raise Exception('hadronstr must be one of nnu, nnd, nns, nnubar, nndbar, nnsbar')
    lilmod = tf.keras.backend.function(model.get_layer('x').input,
                                       model.get_layer(hadronstr).output)
    return lilmod


def fqp(x, QQ, kperp2avg, kperp, flavor):
    '''
    x and QQ must be iterables of some kind
    '''
    #had = functions_new.Hadron()
    had = functions.Hadron()
    fq = had.pdf(flavor, x, QQ)
    return fq*(1/(np.pi*kperp2avg))*np.exp(-kperp**2/kperp2avg)

def xsivdist(model, x, QQ, kperp2avg, flavor, kperp):
    refDict = {-3: 'nnsbar',
               -2: 'nnubar',
               -1: 'nndbar',
               1: 'nnd',
               2: 'nnu',
               3: 'nns'}
    nnqval = nnq(model, np.array([x]), refDict[flavor])
    hval = h(model, kperp)
    fqpval = fqp([x], [QQ], kperp2avg, kperp, flavor)
    
    #return ((2*nnqval*hval*fqpval)[0, :])*x
    return ((2*nnqval*hval*fqpval)[0, :])


def xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, flavor, kperp):
    results = []
    for i in range(numReplicas):
        t = tf.keras.models.load_model('SIDISmodels/rep' + str(i) + '.h5', 
                                         custom_objects={'A0': A0, 'Quotient': Quotient})
        results.append(list(xsivdist(t, x, QQ, kperp2avg, flavor, kperp)))
    return np.array(results)


def plotSivDistBands(numReplicas, x, QQ, kperp2avg, flavor, kperp, numSigma=2):
    #datann = DataANN()
    #X, y, err = datann.makeData(df, [hadron], [dependence])

    results = xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, 2, kperp)
    yhat = results.mean(axis=0)
    yerr = results.std(axis=0)

    plt.fill_between(kperp, yhat-numSigma*yerr, yhat+numSigma*yerr,
                     facecolor='b', alpha=0.3)
    plt.plot(kperp, yhat, 'b', label='$u$')
    
    results = xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, 1, kperp)
    yhat = results.mean(axis=0)
    yerr = results.std(axis=0)
    
    plt.fill_between(kperp, yhat-numSigma*yerr, yhat+numSigma*yerr,
                     facecolor='r', alpha=0.3)
    plt.plot(kperp, yhat, 'r', label='$d$')
    
    results = xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, 3, kperp)
    yhat = results.mean(axis=0)
    yerr = results.std(axis=0)
    
    plt.fill_between(kperp, yhat-numSigma*yerr, yhat+numSigma*yerr,
                     facecolor='g', alpha=0.3)
    plt.plot(kperp, yhat, 'g', label='$s$')
    
    #plt.legend()
    #plt.title('Hermes 2009')
    #plt.xlabel('$k_{\perp}$')
    #plt.ylabel('$x\Delta^Nf(x, k_{\perp})$')
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.ylim(-0.06,0.06)
    plt.legend(loc=4,fontsize=20,handlelength=3)
    plt.savefig('SiversQ_SIDIS_NN_10rep.pdf', format='pdf', bbox_inches='tight')
    
    
def plotSivDistBandsSea(numReplicas, x, QQ, kperp2avg, flavor, kperp, numSigma=2):
    #datann = DataANN()
    #X, y, err = datann.makeData(df, [hadron], [dependence])

    results = xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, -2, kperp)
    yhat = results.mean(axis=0)
    yerr = results.std(axis=0)

    plt.fill_between(kperp, yhat-numSigma*yerr, yhat+numSigma*yerr,
                     facecolor='b', alpha=0.3)
    plt.plot(kperp, yhat, 'b', label='$\\bar{u}$')
    
    results = xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, -1, kperp)
    yhat = results.mean(axis=0)
    yerr = results.std(axis=0)
    
    plt.fill_between(kperp, yhat-numSigma*yerr, yhat+numSigma*yerr,
                     facecolor='r', alpha=0.3)
    plt.plot(kperp, yhat, 'r', label='$\\bar{d}$')
    
    results = xsivdistFromReplicas(numReplicas, x, QQ, kperp2avg, -3, kperp)
    yhat = results.mean(axis=0)
    yerr = results.std(axis=0)
    
    plt.fill_between(kperp, yhat-numSigma*yerr, yhat+numSigma*yerr,
                     facecolor='g', alpha=0.3)
    plt.plot(kperp, yhat, 'g', label='$\\bar{s}$')
    
#     plt.legend()
#     plt.title('Hermes 2009')
#     plt.xlabel('$k_{\perp}$')
#     plt.ylabel('$x\Delta^Nf(x, k_{\perp})$')
#     plt.savefig('2009HermesUDSsea.pdf', format='pdf', bbox_inches='tight')
    #plt.figure(figsize=(15,10))
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.ylim(-0.006,0.006)
    plt.legend(loc=1,fontsize=20,handlelength=3)
    plt.savefig('SiversAntiQ_SIDIS_NN_10rep.pdf', format='pdf', bbox_inches='tight')    

In [4]:
plotSivDistBands(1, 0.1, 2.4, .57, -2, np.array(list(range(150)))/100)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='input_13'), name='input_13', description="created by layer 'input_13'") at layer "dense_36". The following previous layers were accessed without issue: []

In [23]:
tt = tf.keras.models.load_model('SIDISmodels/rep1.h5', custom_objects={'A0': A0, 'Quotient': Quotient})

def xsivdist_test(model, x, QQ, kperp2avg, flavor, kperp):
    refDict = {-3: 'nnsbar',
               -2: 'nnubar',
               -1: 'nndbar',
               1: 'nnd',
               2: 'nnu',
               3: 'nns'}
    nnqval = nnq(model, np.array([x]), refDict[flavor])
    #hval = h(model, kperp)
    #fqpval = fqp([x], [QQ], kperp2avg, kperp, flavor)    
    #return ((2*nnqval*hval*fqpval)[0, :])*x
    #return ((2*nnqval*hval*fqpval)[0, :])
    return nnqval((nnqval)[0, :])

xsivdist_test(tt, 0.2, 2, 1, 2, 1)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='input_13'), name='input_13', description="created by layer 'input_13'") at layer "dense_36". The following previous layers were accessed without issue: []

In [25]:
tt = tf.keras.models.load_model('SIDISmodels/rep1.h5', custom_objects={'A0': A0, 'Quotient': Quotient})
tst_x = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
def nnq_test(model,x, hadronstr):
    if not hadronstr in ['nnu', 'nnd', 'nns', 'nnubar', 'nndbar', 'nnsbar']:
        raise Exception('hadronstr must be one of nnu, nnd, nns, nnubar, nndbar, nnsbar')
    print(model.input)
    print(model.get_layer(hadronstr).output)
    #lilmod = tf.keras.backend.function(model.get_layer('x').input, model.get_layer(hadronstr).output)
    lilmod = tf.keras.backend.function(model.input, model.get_layer(hadronstr).output)
    return lilmod(x)

#tst_x = np.array([0.1,0.2,0.3])
nnq_test(tt,tst_x,'nnu')

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'x')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'z')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'phT')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'uexpr')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'ubarexpr')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dexpr')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dbarexpr')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sexpr')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sbarexpr')>]
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_38/BiasAdd:0', description="created by layer 'dense_38'")


ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='input_13'), name='input_13', description="created by layer 'input_13'") at layer "dense_36". The following previous layers were accessed without issue: []

In [14]:
tf.keras.utils.plot_model(tt, show_shapes=True, show_dtype=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [15]:
tt.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  [(None, 1)]          0                                            
__________________________________________________________________________________________________
nnu (Functional)                (None, 1)            141         x[0][0]                          
__________________________________________________________________________________________________
nnubar (Functional)             (None, 1)            141         x[0][0]                          
__________________________________________________________________________________________________
nnd (Functional)                (None, 1)            141         x[0][0]                          
____________________________________________________________________________________________

In [9]:
tt.get_layer('nnu')

In [30]:
tt.get_layer('nnu').output[0, :]

<KerasTensor: shape=(1,) dtype=float32 (created by layer 'tf.__operators__.getitem_2')>

In [20]:
np.shape(tt.get_layer('nnu').output)

TensorShape([None, 1])

In [22]:
tt = tf.keras.models.load_model('SIDISmodels/rep1.h5', custom_objects={'A0': A0, 'Quotient': Quotient})

In [38]:
tf.keras.backend.function(tt.get_layer('x').input[0, :], tt.get_layer('nnu').output[0, :])

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: tf.__operators__.getitem_5/strided_slice:0


ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name='input_13'), name='input_13', description="created by layer 'input_13'") at layer "dense_36". The following previous layers were accessed without issue: []